In [1]:
!pip install setuptools==58.2.0 packaging==23.2
!pip install python-terrier==0.12.1
!pip install pandas
!pip install torch==2.6.0+cu118 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_colbert.git
!pip install --upgrade git+https://github.com/cmacdonald/pyterrier_bert.git
!pip install faiss-cpu --no-cache-dir

  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
   ---------------------------------------- 0.0/946.1 kB ? eta -:--:--
   --------------------------------------- 946.1/946.1 kB 11.0 MB/s eta 0:00:00
Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 65.5.0
    Uninstalling setuptools-65.5.0:
      Successfully uninstalled setuptools-65.5.0
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
  Using cached python_terrier-0.12.1-py3-none-any.whl.metadata (11 kB)
  Using cached numpy-2.2.4-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached more_itertools-10.6.0-py3-none-any.whl.metadata (37 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadat

  Running command git clone --filter=blob:none --quiet https://github.com/terrierteam/pyterrier_colbert.git 'C:\Users\Konstantin-Asen\AppData\Local\Temp\pip-req-build-2p94_165'
  Running command git clone --filter=blob:none --quiet https://github.com/cmacdonald/ColBERT.git 'C:\Users\Konstantin-Asen\AppData\Local\Temp\pip-install-4gvd2k8m\colbert_e0bd11c855a64a8a89dc00a45193bf4c'
  Running command git checkout -b v0.2 --track origin/v0.2
  branch 'v0.2' set up to track 'origin/v0.2'.
  Switched to a new branch 'v0.2'


  Cloning https://github.com/cmacdonald/pyterrier_bert.git to c:\users\konstantin-asen\appdata\local\temp\pip-req-build-haivyjal
  Resolved https://github.com/cmacdonald/pyterrier_bert.git to commit d75e1d376e8183148587fa584fedb414906e4861
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Cloning https://github.com/Georgetown-IR-Lab/cedr.git to c:\users\konstantin-asen\appdata\local\temp\pip-install-yklpxvx_\cedr_bc3da914c4a346cfaf68cd576b6fc4c3
  Resolved https://github.com/Georgetown-IR-Lab/cedr.git to commit a263f1e00b4b4dcfdc1aa1519793224138be44fc
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting r

  Running command git clone --filter=blob:none --quiet https://github.com/cmacdonald/pyterrier_bert.git 'C:\Users\Konstantin-Asen\AppData\Local\Temp\pip-req-build-haivyjal'
  Running command git clone --filter=blob:none --quiet https://github.com/Georgetown-IR-Lab/cedr.git 'C:\Users\Konstantin-Asen\AppData\Local\Temp\pip-install-yklpxvx_\cedr_bc3da914c4a346cfaf68cd576b6fc4c3'
  Running command git clone --filter=blob:none --quiet https://github.com/terrier-org/pyterrier.git 'C:\Users\Konstantin-Asen\AppData\Local\Temp\pip-install-yklpxvx_\python-terrier_d445769226024a09819af6daf03a3fff'
ERROR: Could not find a version that satisfies the requirement torch==1.4.0 (from pyterrierbert) (from versions: 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0)
ERROR: No matching distribution found for torch==1.4.0


   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   ------------- -------------------------- 4.7/13.7 MB 28.6 MB/s eta 0:00:01
   --------------------------------- ------ 11.5/13.7 MB 30.1 MB/s eta 0:00:01
   ---------------------------------------- 13.7/13.7 MB 30.7 MB/s eta 0:00:00


The code below preferably should print True before running the experiments; if it's True, it will run faster

In [10]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
1


In [11]:
import pyterrier as pt
import pandas as pd

In [12]:
msmarco = pt.get_dataset('msmarco_passage')
dataset = list(msmarco.get_corpus_iter())[:1000] # SMALL DATASET

print("First document:", dataset[0] if len(dataset) > 0 else "Dataset is empty!")

First document: {'docno': '0', 'text': 'The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.\n'}


In [13]:
import os

marco_index_src=os.path.abspath("msmarco-index")
if not os.path.exists(marco_index_src):
    print("Index not found. Creating a new index...")
    # Create index with metadata
    indexer = pt.IterDictIndexer(marco_index_src, blocks=True)

    index_ref = indexer.index(dataset, fields=["text"])
    
    # TOGGLE FOR FULL DATASET (below)
    # index_ref = indexer.index(msmarco.get_corpus_iter(), fields=["text"])

    print("Index successfully created!")

index = pt.IndexFactory.of(marco_index_src)

In [14]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
rm3 = pt.rewrite.RM3(index)

C:\Users\Konstantin-Asen\AppData\Local\Temp\ipykernel_4172\959554160.py:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index, wmodel="BM25")


In [15]:
queries = msmarco.get_topics("dev.small")[:1000]

# TOGGLE FOR FULL DATASET (below)
# queries = msmarco.get_topics("dev.small")

print(f"Checking Queries:\n{queries.head()}")

qrels = msmarco.get_qrels("dev.small")[:1000]

# TOGGLE FOR FULL DATASET (below)
# qrels = msmarco.get_qrels("dev.small")

print(f"Checking Result Relevance:\n{qrels.head()}")

Checking Queries:
       qid                                          query
0  1048585                   what is paula deen s brother
1        2                       androgen receptor define
2   524332  treating tension headaches without medication
3  1048642                            what is paranoid sc
4   524447            treatment of varicose veins in legs
Checking QRels:
      qid    docno  label
0  300674  7067032      1
1  125705  7067056      1
2   94798  7067181      1
3    9083  7067274      1
4  174249  7067348      1


In [16]:
bm25(queries)

,qid,docid,docno,rank,score,query
0,1048585,11,11,0,8.998658,what is paula deen s brother
1,1048585,44,44,1,7.937727,what is paula deen s brother
2,1048585,245,245,2,6.830438,what is paula deen s brother
3,1048585,573,573,3,6.661482,what is paula deen s brother
4,1048585,623,623,4,5.340478,what is paula deen s brother
...,...,...,...,...,...,...
289029,1048565,457,457,78,3.066126,who plays sebastian michaelis
289030,1048565,426,426,79,3.033457,who plays sebastian michaelis
289031,1048565,96,96,80,3.001477,who plays sebastian michaelis
289032,1048565,659,659,81,2.395397,who plays sebastian michaelis


In [17]:
rm3(bm25(queries))

,qid,query_0,query
0,1000000,where does real insulin come from,applypipeline:off life^0.026955701 video^0.052...
1,1000004,where does name nora come from,applypipeline:off window^0.036981232 applianc^...
2,1000006,where does most of the iron ore come from,applypipeline:off account^0.024999995 product^...
3,1000017,where does misery take place,applypipeline:off page^0.033703115 take^0.2477...
4,1000030,where does microtubule formation occur,applypipeline:off md^0.017548742 evolut^0.0344...
...,...,...,...
6433,999791,where does tulips now grow,applypipeline:off now^0.200000018 colorado^0.0...
6434,999836,where does the signature block go on a memorandum,applypipeline:off work^0.035556179 dna^0.07111...
6435,999897,where is way st binghamton,applypipeline:off st^0.235996157 8^0.026546501...
6436,999921,define shore up,applypipeline:off area^0.032073446 beauti^0.03...


In [19]:
from pyterrier.measures import MAP, nDCG

pt.Experiment(
    [bm25, bm25 >> rm3 >> bm25],
    queries,
    qrels,
    names=["BM25", "BM25 >> RM3 >> BM25"],
    eval_metrics=[MAP, nDCG @ 10],
)

,name,AP,nDCG@10
0,BM25,0.0,0.0
1,BM25 >> RM3 >> BM25,0.0,0.0


In [20]:
from pyterrier_colbert.indexing import ColBERTIndexer
from pyterrier_colbert.ranking import ColBERTFactory
import wget
import zipfile

checkpoint_url = "http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip"
extract_dir = "colbert_checkpoint"
checkpoint_path = "colbert_checkpoint.zip"

if not os.path.exists(checkpoint_path):
    print("Downloading checkpoint...")
    wget.download(checkpoint_url, checkpoint_path)
if not os.path.exists(extract_dir):
    with zipfile.ZipFile(checkpoint_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

colbert_checkpoint_path = os.path.abspath("colbert_checkpoint/colbert.dnn")
index_root = os.path.abspath("msmarco-index")
index_name = os.path.abspath("colbert-index")

colbert_index = ColBERTIndexer(
    checkpoint=colbert_checkpoint_path,
    index_root=index_root,
    index_name=index_name,
    chunksize=64,
    gpu=True # if the torch.cuda returned False, comment this
)
colbert_index.index(dataset)

# TOGGLE FOR FULL DATASET (below)
# colbert_index.index(msmarco.get_corpus_iter())

c:\Users\Konstantin-Asen\Desktop\notebook-workspace\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Mar 20, 17:49:09] [0] 		 #> Local args.bsize = 128
[Mar 20, 17:49:09] [0] 		 #> args.index_root = c:\Users\Konstantin-Asen\Desktop\notebook-workspace\msmarco-index
[Mar 20, 17:49:09] [0] 		 #> self.possible_subset_sizes = [1491308]


Some weights of ColBERT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['linear.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Mar 20, 17:49:11] #> Loading model checkpoint.
[Mar 20, 17:49:11] #> Loading checkpoint c:\Users\Konstantin-Asen\Desktop\notebook-workspace\colbert_checkpoint\colbert.dnn
[Mar 20, 17:49:12] #> checkpoint['epoch'] = 0
[Mar 20, 17:49:12] #> checkpoint['batch'] = 44500




c:\Users\Konstantin-Asen\Desktop\notebook-workspace\.venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Konstantin-Asen\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)




[Mar 20, 17:49:14] #> Note: Output directory c:\Users\Konstantin-Asen\Desktop\notebook-workspace\msmarco-index already exists




[Mar 20, 17:49:14] #> Creating directory c:\Users\Konstantin-Asen\Desktop\notebook-workspace\colbert-index 




In [ ]:
pytcolbert = ColBERTFactory(colbert_checkpoint_path, index_root, index_name)

In [ ]:
dense_e2e = pytcolbert.end_to_end()

In [ ]:
sparse_colbert = bm25 >> pytcolbert.text_scorer()

In [ ]:
pt.Experiment(
    [bm25, sparse_colbert, dense_e2e],
    queries,
    qrels,
    names=["BM25", "BM25 >> ColBERT", "Dense ColBERT"],
    eval_metrics=[MAP, nDCG @ 10]
)